In [2]:
# download data from IMDb website and retain only .gz files
#!wget -A gz -m -p -E -k -K -nd -e robots=off https://datasets.imdbws.com/
!cp data/* ./
!rm ./*.tsv

# list all data files
import os
os.listdir()

rm: ./*.tsv: No such file or directory


['title.principals.tsv.gz',
 'title.akas.tsv.gz',
 'title.basics.tsv.gz',
 'title.crew.tsv.gz',
 'README.md',
 'title.ratings.tsv.gz',
 '.gitignore',
 'IMDb-Rating-Prediction_ETL.ipynb',
 '.ipynb_checkpoints',
 'name.basics.tsv.gz',
 '.git',
 'data',
 'title.episode.tsv.gz']

In [3]:
import pandas as pd
import numpy as np

# function to unzip a given tsv.gz file and read into pd.Dataframe
def import_df(tsv_file_name):
    !gzip -d {tsv_file_name}.gz
    df = pd.read_csv(tsv_file_name, sep='\t', header=0, dtype="string")
    return df

# process title.basics.tsv
df_title_basics = import_df("title.basics.tsv").set_index("tconst")
df_title_basics.head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
8,tt0000009,short,Miss Jerry,Miss Jerry,0,1894,\N,40,"Romance,Short"
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [4]:
# There are many complications for TV episodes and other titleTypes, we will focus on movies for this project
# convert types:
types_map = {"isAdult": "float", "startYear": "float", "runtimeMinutes": "float", "tconst": "string", "primaryTitle": "string", "originalTitle": "string"}
int_map = {"isAdult": "Int32", "startYear": "Int32", "runtimeMinutes": "Int32"}

# endYear is not meaningful for movies, drop it
# don't need titleType any more since we have only movies
# convert \N values to np.nan for further processing
df_movies_basics = df_title_basics[df_title_basics["titleType"]=="movie"].applymap(lambda x : np.nan if x == "\\N" else x).drop(columns=["endYear", "titleType"]).astype(types_map).astype(int_map)
# show first 10 rows
df_movies_basics.head(10)

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
498,tt0000502,Bohemios,Bohemios,0,1905,100,NaN
570,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography"
587,tt0000591,The Prodigal Son,L'enfant prodigue,0,1907,90,Drama
610,tt0000615,Robbery Under Arms,Robbery Under Arms,0,1907,<NA>,Drama
625,tt0000630,Hamlet,Amleto,0,1908,<NA>,Drama
668,tt0000675,Don Quijote,Don Quijote,0,1908,<NA>,Drama
672,tt0000679,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,120,"Adventure,Fantasy"
729,tt0000739,El pastorcito de Torrente,El pastorcito de Torrente,0,1908,<NA>,Drama
783,tt0000793,Andreas Hofer,Andreas Hofer,0,1909,<NA>,Drama
804,tt0000814,La bocana de Mar Chica,La bocana de Mar Chica,0,1909,<NA>,NaN


In [10]:
df_movies_basics = df_movies_basics.set_index("tconst")

In [6]:
df_movies_basics["isAdult"].value_counts()

0    579290
1      9058
Name: isAdult, dtype: Int64

In [8]:
df_title_ratings = import_df("title.ratings.tsv").set_index("tconst")
df_title_ratings.head(10)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1818
1,tt0000002,6.0,232
2,tt0000003,6.5,1574
3,tt0000004,6.1,151
4,tt0000005,6.2,2399
5,tt0000006,5.2,157
6,tt0000007,5.4,747
7,tt0000008,5.5,1974
8,tt0000009,5.9,190
9,tt0000010,6.9,6550


In [58]:
def expandCol(df, colName, sep):
    splitted = df[colName].str.split(sep)
    for val in splitted.explode(colName).value_counts().keys():
        df[val] = df[colName].str.contains(val)
        df[val].fillna(False)
        

In [59]:
df_movie_ratings_basics = df_title_ratings.join(df_movies_basics, how="inner")
expandCol(df_movie_ratings_basics, "genres", ',')

In [60]:
df_movie_ratings_basics

,averageRating,numVotes,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,Drama,Comedy,...,Animation,Western,Adult,Sport,Film-Noir,News,Reality-TV,Short,Talk-Show,Game-Show
tconst,,,,,,,,,,,,,,,,,,,,,
tt0000502,4.5,14,Bohemios,Bohemios,0,1905,100,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000574,6.1,694,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",False,False,...,False,False,False,False,False,False,False,False,False,False
tt0000591,5.2,16,The Prodigal Son,L'enfant prodigue,0,1907,90,Drama,True,False,...,False,False,False,False,False,False,False,False,False,False
tt0000615,4.5,23,Robbery Under Arms,Robbery Under Arms,0,1907,<NA>,Drama,True,False,...,False,False,False,False,False,False,False,False,False,False
tt0000630,3.8,23,Hamlet,Amleto,0,1908,<NA>,Drama,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9916190,3.6,222,Safeguard,Safeguard,0,2020,90,"Action,Adventure,Thriller",False,False,...,False,False,False,False,False,False,False,False,False,False
tt9916270,5.8,1176,Il talento del calabrone,Il talento del calabrone,0,2020,84,Thriller,False,False,...,False,False,False,False,False,False,False,False,False,False
tt9916362,6.4,3888,Coven,Akelarre,0,2020,92,"Drama,History",True,False,...,False,False,False,False,False,False,False,False,False,False
